<a href="https://colab.research.google.com/github/SoojinY/thesesWordCloud/blob/main/wordCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://jehyunlee.github.io/2021/08/13/Python-DS-82-scopusapi/

https://www.kci.go.kr/kciportal/po/openapi/openApiConnView.kci

https://wonhwa.tistory.com/9 


# KCI 국내학술지 인용색인 정보 포탈

https://www.kci.go.kr/kciportal/po/openapi/openApiKeyRequestView.kci

In [50]:
# 라이브러리 import
import requests # API 요청 보내기 위함
import pprint # xml 출력 확인
import urllib # url 관련 작업

In [52]:
# url 입력
key =  # KCI API 키
searchWord = '코로나' # 검색 단어
searchWord = urllib.parse.quote_plus(searchWord) # 검색 단어 URL 인코딩
count = str(100) # 검색 논문 수. KCI API 는 기본 10, 최대 100 까지 제공

url = 'https://open.kci.go.kr/po/openapi/openApiSearch.kci?apiCode=articleSearch&key=' + key + '&keyword=' + searchWord + '&displayCount=' + count 

# url 불러오기
response = requests.get(url, verify=False)
contents = response.text

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# XML -> DataFame 변환 

https://wonhwa.tistory.com/16

In [ ]:
# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(contents))

https://www.crummy.com/software/BeautifulSoup/bs4/doc.ko/

뷰티플수프 문서

필요한 정보를 반환 받기 위해 참고

In [3]:
### xml을 DataFrame으로 변환하기 ###
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

In [54]:
xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
rows = xml_obj.findAll('record')
print(rows[0])

<record>
<journalInfo>
<journal-name>한국사전학</journal-name>
<publisher-name>한국사전학회</publisher-name>
<pub-year>2020</pub-year>
<pub-mon>11</pub-mon>
<volume/>
<issue>36</issue>
</journalInfo>
<articleInfo article-id="ART002656882">
<article-categories>사전학</article-categories>
<article-regularity>Y</article-regularity>
<title-group>
<article-title lang="original">코로나-19 신어의 수집과 사용 양상 연구 - 주제 특정적 신어의 수집과 사용에 대한 고찰 -</article-title>
<article-title lang="foreign">COVID-19 Neologism Collection and Usage Pattern Analysis -Consideration of Topic-Specific Neologism Extraction and Usage Pattern Analysis-</article-title>
<article-title lang="english">COVID-19 Neologism Collection and Usage Pattern Analysis -Consideration of Topic-Specific Neologism Extraction and Usage Pattern Analysis-</article-title>
</title-group>
<author-group>
<author>이수진(경북대학교)</author>
<author>강현아(경북대학교)</author>
<author>남길임(경북대학교)</author>
</author-group>
<abstract-group>
<abstract lang="original">This paper collects and ex

In [55]:
# 분석에 사용할 열 이름 리스트
name_list = [
   'article-title-original'
  ,'article-title-english'
  ,'author'
  ,'abstract-original'
  ,'abstract-english'
  ,'url'          
]

In [56]:
data_list = [] # 데이터 담는 리스트
row_list = [] # 행 담는 리스트

for row in rows:
  col = row.find_all()
  row_list = [0]*6 # row_list 초기화

  for i in range(0, len(col)):
    
    name = col[i].name
    lang = col[i]['lang'] if col[i].has_attr('lang')  else ''
    text = col[i].text
    # name_list의 열에 맞춰 해당하는 text를 row_list에 저장
    if name == 'url': row_list[5] = text
    elif name == 'author' and row_list[2] == 0: row_list[2] = text
    elif name == 'article-title':
      if lang == 'original': row_list[0] = text
      elif lang == 'english': row_list[1] = text
    elif name == 'abstract':
      if lang == 'original': row_list[3] = text
      elif lang == 'english': row_list[4] = text

  data_list.append(row_list) # row_list를 data_list에 저장
  row_list=[] # row_list 비우기


In [57]:
print(len(data_list))
data_list[0]

100


['코로나-19 신어의 수집과 사용 양상 연구 - 주제 특정적 신어의 수집과 사용에 대한 고찰 -',
 'COVID-19 Neologism Collection and Usage Pattern Analysis -Consideration of Topic-Specific Neologism Extraction and Usage Pattern Analysis-',
 '이수진(경북대학교)',
 'This paper collects and examines Korean neologisms related to COVID-19 and analyzes their usage patterns. It also considers the methodology for the study of topic-specific neologisms. In order to achieve this, the paper follows two research frameworks: 1) “from COVID-19 to language”; 2) “from language to COVID-19”. The former examines the impact of COVID-19 on language and explores methods of collecting of COVID-19-related neologisms, for which it was first necessary to define the category of COVID-19 neologisms and discuss their distinguishing criteria. The second research framework explores how language informs us on the COVID-19 situation. The analysis of the time when a given COVID-19 neologism first appeared, the changes in occurrence frequency between January and Jul

In [58]:
# 추출한 정보를 DataFrame으로 만들기
df = pd.DataFrame(data_list, columns=name_list)
df.head(5)

,article-title-original,article-title-english,author,abstract-original,abstract-english,url
0,코로나-19 신어의 수집과 사용 양상 연구 - 주제 특정적 신어의 수집과 사용에 대...,COVID-19 Neologism Collection and Usage Patter...,이수진(경북대학교),This paper collects and examines Korean neolog...,This paper collects and examines Korean neolog...,https://www.kci.go.kr/kciportal/ci/sereArticle...
1,포스트코로나 시대의 동시문학,The Children’s Poetry Literature of the Post-C...,방은수(서울교육대학교),이 연구의 목적은 포스트코로나 시대의 동시문학에 대해 탐색하는것이다. 먼저 감염병이...,The purpose of this study is to explore the ch...,https://www.kci.go.kr/kciportal/ci/sereArticle...
2,코로나19와 사회보장법의 대응 — 핀란드와 한국의 사례 비교 —,Response of the Social Security System to COVI...,김남희(서울대학교 법학전문대학원),"코로나19의 전세계적 확산 이후 각국은 경제, 사회 전반의 위기를 겪고 있으며, 다...","Since the global spread of COVID-19, each coun...",https://www.kci.go.kr/kciportal/ci/sereArticle...
3,코로나 위기 시대와 구약신학의 과제,코로나 위기 시대와 구약신학의 과제,강성열(호남신학대학교),본 논문의 목적은 코로나 위기 시대에 직면하여 구약신학의 과제를 제시하려는 데 있다...,The purpose of this article is to propose the ...,https://www.kci.go.kr/kciportal/ci/sereArticle...
4,신종 코로나바이러스 감염증(COVID-19) 유행의 대응과 치료,Coronavirus Disease 2019 (COVID-19): Pandemic ...,이승화(성남시의료원 가정의학과),중국에서 첫 발병한 신종 코로나바이러스(SARS-CoV-2)로 인한 감염증은 현재 ...,Infectious diseases caused by the novel corona...,https://www.kci.go.kr/kciportal/ci/sereArticle...
